In [1]:
from google.colab import drive
drive.mount('/gdrive')

In [ ]:
%cd /gdrive/My Drive/AN2DL_Assignment02

In [2]:
import tensorflow as tf
import numpy as np
import os
import random
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
plt.rc('font', size=16)
import warnings
warnings.filterwarnings('ignore')
tf.get_logger().setLevel('ERROR')

tfk = tf.keras
tfkl = tf.keras.layers
print(tf.__version__)

In [3]:
# Random seed for reproducibility
seed = 42

random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

In [5]:
# Load the dataset
dataset = pd.read_csv('Training.csv')
print(dataset.shape)

In [6]:
# Function to visualize the time series
def inspect_dataframe(df, columns):
    figs, axs = plt.subplots(len(columns), 1, sharex=True, figsize=(20,27))
    for i, col in enumerate(columns):
        axs[i].plot(df[col])
        axs[i].set_title(col)
    plt.show()
    
inspect_dataframe(dataset, dataset.columns)

In [7]:
# Normalize both features and labels
X_min = dataset.min()
X_max = dataset.max()

X_train_raw = (dataset-X_min)/(X_max-X_min)
# The test set has been discarded for the final training, in order to have the highest number of available training samples

In [8]:
# Definition of the window, the stride and the telescope
window = 500
stride = 1
telescope = 864 # The number of points we want to predict correspond the telescope of phase 1

In [9]:
# Function to build the training sequences
def build_sequences(df, target_labels=['Sponginess'], window=200, stride=20, telescope=100):
    # Sanity check to avoid runtime errors
    assert window % stride == 0
    dataset = []
    labels = []
    temp_df = df.copy().values
    temp_label = df[target_labels].copy().values # Can be one column, a subset of columns, or even the entire dataset
    padding_len = len(df)%window

    if(padding_len != 0):
        # Compute padding length
        padding_len = window - len(df)%window
        padding = np.zeros((padding_len,temp_df.shape[1]), dtype='float64')
        temp_df = np.concatenate((padding,df))
        padding = np.zeros((padding_len,temp_label.shape[1]), dtype='float64')
        temp_label = np.concatenate((padding,temp_label))
        assert len(temp_df) % window == 0

    for idx in np.arange(0,len(temp_df)-window-telescope,stride):
        dataset.append(temp_df[idx:idx+window])
        labels.append(temp_label[idx+window:idx+window+telescope])

    dataset = np.array(dataset)
    labels = np.array(labels)
    return dataset, labels

In [10]:
target_labels = dataset.columns

X_train, y_train = build_sequences(X_train_raw, target_labels, window, stride, telescope)

X_train.shape, y_train.shape

In [11]:
# Function to plot randomly a training sequence and its true prediction
def inspect_multivariate(X, y, columns, telescope, idx=None):
    if(idx==None):
        idx=np.random.randint(0,len(X))

    figs, axs = plt.subplots(len(columns), 1, sharex=True, figsize=(17,17))
    for i, col in enumerate(columns):
        axs[i].plot(np.arange(len(X[0,:,i])), X[idx,:,i])
        axs[i].scatter(np.arange(len(X[0,:,i]), len(X_train[0,:,i])+telescope), y[idx,:,i], color='orange')
        axs[i].set_title(col)
        axs[i].set_ylim(0,1)
    plt.show()
    
inspect_multivariate(X_train, y_train, target_labels, telescope)

In [12]:
input_shape = X_train.shape[1:]
output_shape = y_train.shape[1:]
batch_size = 32
epochs = 200

In [13]:
def build_concat_LSTM_model(input_shape, output_shape):

    input_layer = tfkl.Input(shape=input_shape, name='Input')

    # 1st block
    convlstm = tfkl.Bidirectional(tfkl.LSTM(64, return_sequences=True))(input_layer)
    convlstm = tfkl.Bidirectional(tfkl.LSTM(128))(convlstm)

    # 2nd block
    convlstm1 = tfkl.Bidirectional(tfkl.LSTM(64, return_sequences=True))(input_layer)
    convlstm1 = tfkl.Bidirectional(tfkl.LSTM(128))(convlstm1)

    convlstm = tfkl.concatenate([convlstm, convlstm1])
    convlstm = tfkl.Dropout(.2)(convlstm)
 
    dense = tfkl.Dense(output_shape[-1]*output_shape[-2], activation='relu')(convlstm)
    output_layer = tfkl.Reshape((output_shape[-2],output_shape[-1]))(dense)

    # Connect input and output through the Model class
    model = tfk.Model(inputs=input_layer, outputs=output_layer, name='model')

    # Compile the model
    model.compile(loss=tfk.losses.MeanSquaredError(), optimizer=tfk.optimizers.Adam(1e-4), metrics=['mae',tf.keras.metrics.RootMeanSquaredError()])

    # Return the model
    return model

In [14]:
# Build and plot the model
model = build_concat_LSTM_model(input_shape, output_shape)
model.summary()
tfk.utils.plot_model(model, expand_nested=True)

In [15]:
# Train the model
history = model.fit(x = X_train,
                    y = y_train,
                    batch_size = batch_size,
                    epochs = epochs,
                    validation_split=0.05,
                    callbacks = [tfk.callbacks.EarlyStopping(monitor='val_root_mean_squared_error', mode='min', patience=5, restore_best_weights=True),
                                 tfk.callbacks.ReduceLROnPlateau(monitor='val_root_mean_squared_error', mode='min', patience=5, factor=0.5, min_lr=1e-5)]
                    ).history

In [16]:
# Save best epoch model
model.save('Concat_Bi_LSTM_model')

In [17]:
# Plot the training
best_epoch = np.argmin(history['val_loss'])
plt.figure(figsize=(17,4))
plt.plot(history['loss'], label='Training loss', alpha=.8, color='#ff7f0e')
plt.plot(history['val_loss'], label='Validation loss', alpha=.9, color='#5a9aa5')
plt.axvline(x=best_epoch, label='Best epoch', alpha=.3, ls='--', color='#5a9aa5')
plt.title('Mean Squared Error (Loss)')
plt.legend()
plt.grid(alpha=.3)
plt.show()

plt.figure(figsize=(17,4))
plt.plot(history['mae'], label='Training accuracy', alpha=.8, color='#ff7f0e')
plt.plot(history['val_mae'], label='Validation accuracy', alpha=.9, color='#5a9aa5')
plt.axvline(x=best_epoch, label='Best epoch', alpha=.3, ls='--', color='#5a9aa5')
plt.title('Mean Absolute Error')
plt.legend()
plt.grid(alpha=.3)
plt.show()

plt.figure(figsize=(18,3))
plt.plot(history['lr'], label='Learning Rate', alpha=.8, color='#ff7f0e')
plt.axvline(x=best_epoch, label='Best epoch', alpha=.3, ls='--', color='#5a9aa5')
plt.legend()
plt.grid(alpha=.3)
plt.show()

In [18]:
future = dataset[-window:]
future = (future-X_min)/(X_max-X_min)
future = np.expand_dims(future, axis=0)

In [19]:
# Predict the future 864 points
future_predictions = model.predict(future)

In [21]:
# Visualize the prediction
figs, axs = plt.subplots(len(target_labels), 1, sharex=True, figsize=(17,17))
for i, col in enumerate(target_labels):
    axs[i].plot(np.arange(len(future[0,:,i])), future[0,:,i])
    axs[i].plot(np.arange(len(future[0,:,i]), len(future[0,:,i])+telescope), future_predictions[0,:,i], color='orange')
    axs[i].set_title(col)
plt.show()